In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import cv2

In [4]:
from datetime import datetime

In [5]:
import os

In [6]:
import face_recognition

In [7]:
# Directories for student images and group photos
individual_images_dir = r'C:\Users\NIKHAR BEHERA\Anaconda\Face_advr\indi_img'
group_images_dir = r'C:\Users\NIKHAR BEHERA\Anaconda\Face_advr\grp_img'

In [8]:
# Load and encode student images
def load_and_encode_images(directory):
    known_encodings = []
    student_names = []
    
    for filename in os.listdir(directory):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            # Load image
            image_path = os.path.join(directory, filename)
            img = face_recognition.load_image_file(image_path)
            
            # Handle cases with no faces or multiple faces in the image
            try:
                img_encoding = face_recognition.face_encodings(img)[0]
                known_encodings.append(img_encoding)
                student_names.append(os.path.splitext(filename)[0])  # Use the filename without extension as the name
            except IndexError:
                print(f"Warning: No face found in {filename}")
    
    return known_encodings, student_names


In [9]:
# Recognize faces in group photo
def recognize_faces_in_group_photo(image_path, known_encodings, student_names, tolerance=0.5):
    # Load the group image
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Detect faces in the group image
    face_locations = face_recognition.face_locations(img_rgb)
    face_encodings = face_recognition.face_encodings(img_rgb, face_locations)
    
    recognized_names = []
    
    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_encodings, face_encoding, tolerance=tolerance)
        face_distances = face_recognition.face_distance(known_encodings, face_encoding)
        best_match_index = None
        
        # Find the best match index with a stricter check
        if len(face_distances) > 0:
            best_match_index = face_distances.argmin()
            if matches[best_match_index] and face_distances[best_match_index] <= tolerance:
                name = student_names[best_match_index]
            else:
                name = "Unknown"
        else:
            name = "Unknown"
        
        recognized_names.append(name)
    
    return recognized_names, face_locations


In [10]:
# Mark attendance in an Excel file
def mark_attendance(names, output_file=r'C:\Users\NIKHAR BEHERA\Anaconda\Face_advr\attendance.xlsx'):
    # Load the existing attendance file or create a new one
    try:
        df = pd.read_excel(output_file)
    except FileNotFoundError:
        df = pd.DataFrame(columns=['Name', 'Time'])

    # Get the current timestamp
    now = datetime.now()
    dt_string = now.strftime('%Y-%m-%d %H:%M:%S')

    # Add recognized names to the attendance list
    new_entries = pd.DataFrame({'Name': names, 'Time': dt_string})
    df = pd.concat([df, new_entries[~new_entries['Name'].isin(df['Name'])]], ignore_index=True)

    # Save the updated attendance sheet
    df.to_excel(output_file, index=False)


In [11]:
# Draw bounding boxes on recognized faces
def draw_bounding_boxes(img, face_locations, recognized_names):
    for (top, right, bottom, left), name in zip(face_locations, recognized_names):
        # Set the bounding box color based on whether the face is recognized
        color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
        
        # Draw a rectangle around the face
        cv2.rectangle(img, (left, top), (right, bottom), color, 2)
        
        # Add the student's name
        font_scale = 0.4 if name != "Unknown" else 0.6  # Adjust font size for visibility
        cv2.putText(img, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, 2)


In [12]:
# Process group photos and mark attendance
def process_group_photos():
    # Load student images and encode faces
    known_encodings, student_names = load_and_encode_images(individual_images_dir)
    
    # Process each group photo in the directory
    for filename in os.listdir(group_images_dir):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(group_images_dir, filename)
            recognized_names, face_locations = recognize_faces_in_group_photo(image_path, known_encodings, student_names)
            
            # Mark attendance
            mark_attendance(recognized_names)
            
            # Draw bounding boxes around recognized faces
            img = cv2.imread(image_path)
            draw_bounding_boxes(img, face_locations, recognized_names)
            
            # Display the processed image
            cv2.imshow('Attendance', img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()


In [13]:
# Run the attendance marking process
if __name__ == "__main__":
    process_group_photos()
